# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession
session = SparkSession.builder.getOrCreate()

In [3]:
session

In [5]:
session.sparkContext.getConf().getAll()

[('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '10.0.128.143'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.id', 'local-1557567979563'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '58377')]

#### Passing other options to spark session:
    
    

In [6]:
session = SparkSession.builder.config('someoption.key','somevalue').getOrCreate()

We can check option values in the resulting session like this:

In [7]:
session.sparkContext.getConf().getAll()

[('spark.sql.catalogImplementation', 'hive'),
 ('someoption.key', 'somevalue'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '10.0.128.143'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1557567979563'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '58377')]

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [8]:
import random
random.choice(['a','b','c'])

'b'

In [9]:
random.seed(42)
ids = range(12)
positions = [random.choice(['mecanic','sales','manager']) for id_ in ids]
positions

['manager',
 'mecanic',
 'mecanic',
 'manager',
 'sales',
 'mecanic',
 'mecanic',
 'mecanic',
 'manager',
 'mecanic',
 'manager',
 'manager']

In [10]:
rows = zip(ids,positions)
df = session.createDataFrame(rows)
df

DataFrame[_1: bigint, _2: string]

In [13]:
df.take(5)

[Row(_1=0, _2='manager'),
 Row(_1=1, _2='mecanic'),
 Row(_1=2, _2='mecanic'),
 Row(_1=3, _2='manager'),
 Row(_1=4, _2='sales')]

In [14]:
df.show(5)

+---+-------+
| _1|     _2|
+---+-------+
|  0|manager|
|  1|mecanic|
|  2|mecanic|
|  3|manager|
|  4|  sales|
+---+-------+
only showing top 5 rows



In [15]:
from pyspark.sql import Row

In [16]:
help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  A row in L{DataFrame}.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments,
 |  the fields will be sorted by names. It is not allowed to omit
 |  a named argument to represent the value is None or missing. This should be
 |  explicitly set to None in this case.
 |  
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(age=11, name='Alice')
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 |  True
 |  >>> 'wrong_key' in row
 |  False
 |  
 |  Row also can be used to create another Row like class, then it
 |  could be used to create Row objects, such as
 |  
 |  >>> Person = Row("name", "age")
 |  >>> Person
 |  <Row(name, age)>
 |  >>> 'name' 

In [17]:
rows = [Row(id=id_, position=position_) for id_,position_ in zip(ids,positions)]

In [18]:
rows

[Row(id=0, position='manager'),
 Row(id=1, position='mecanic'),
 Row(id=2, position='mecanic'),
 Row(id=3, position='manager'),
 Row(id=4, position='sales'),
 Row(id=5, position='mecanic'),
 Row(id=6, position='mecanic'),
 Row(id=7, position='mecanic'),
 Row(id=8, position='manager'),
 Row(id=9, position='mecanic'),
 Row(id=10, position='manager'),
 Row(id=11, position='manager')]

In [19]:
df = session.createDataFrame(rows)
df.show(5)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1| mecanic|
|  2| mecanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



In [29]:
myDf = session.createDataFrame(zip(ids,positions),schema=['id','position'])

In [31]:
myDf.show(5)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1| mecanic|
|  2| mecanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



In [23]:
myRdd = df.rdd

In [24]:
myRdd.take(5)

[Row(id=0, position='manager'),
 Row(id=1, position='mecanic'),
 Row(id=2, position='mecanic'),
 Row(id=3, position='manager'),
 Row(id=4, position='sales')]

### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [33]:
rdd = session.sparkContext.textFile('coupon150720.csv')
rdd

coupon150720.csv MapPartitionsRDD[62] at textFile at NativeMethodAccessorImpl.java:0

In [34]:
rdd.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [35]:
split_rdd=rdd.map(lambda line: line.split(','))
split_rdd.take(2)

[['79062005698500',
  '1',
  'MAA',
  'AUH',
  '9W',
  '9W',
  '56.79',
  'USD',
  '1',
  'H',
  'H',
  '0526',
  '150904',
  'OK',
  'IAF0'],
 ['79062005698500',
  '2',
  'AUH',
  'CDG',
  '9W',
  '9W',
  '84.34',
  'USD',
  '1',
  'H',
  'H',
  '6120',
  '150905',
  'OK',
  'IAF0']]

### Inferring and specifying schemas

In [36]:
new_df = session.createDataFrame(split_rdd)
new_df.show(5)

+--------------+---+---+---+---+---+------+---+---+---+---+----+------+---+----+
|            _1| _2| _3| _4| _5| _6|    _7| _8| _9|_10|_11| _12|   _13|_14| _15|
+--------------+---+---+---+---+---+------+---+---+---+---+----+------+---+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|  H|0526|150904| OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|  H|6120|150905| OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|  H|2768|150721| OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|  S|0546|150804| OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|  V|0501|150803| OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+---+----+------+---+----+
only showing top 5 rows



In [37]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)



In [39]:
new_df.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)
 |-- _7: string (nullable = true)
 |-- _8: string (nullable = true)
 |-- _9: string (nullable = true)
 |-- _10: string (nullable = true)
 |-- _11: string (nullable = true)
 |-- _12: string (nullable = true)
 |-- _13: string (nullable = true)
 |-- _14: string (nullable = true)
 |-- _15: string (nullable = true)



#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [41]:
from pyspark.sql import types
types.IntegerType()

IntegerType

In [42]:
fields = [types.StructField('id', types.IntegerType()),
         types.StructField('position',types.StringType())]
my_schema = types.StructType(fields)
my_schema

StructType(List(StructField(id,IntegerType,true),StructField(position,StringType,true)))

In [43]:
session.createDataFrame(zip(ids,positions),schema=my_schema)

DataFrame[id: int, position: string]

#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [44]:
df_from_csv = session.read.csv('coupon150720.csv')

In [45]:
df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [47]:
df_with_sql = session.sql('SELECT * from csv.`coupon150720.csv` where _c3="MAD"')

In [49]:
df_with_sql.show(5)

+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|79062005639642|  4|BRU|MAD| UX| UX|21.02|USD|  2|  Q|   Q|1172|150912|  OK|INBQ|
|79065668754871|  6|BRU|MAD| SN| 9W|27.66|USD|  2|  W|   W|6021|151018|  OK|INBQ|
|79065668917696|  2|CDG|MAD| AF| AF|46.97|USD|  1|  T|   T|1400|150905|  OK|INB0|
|79062006133090|  2|CDG|MAD| AF| AF| 3.38|USD|  1|  X|   X|1400|151103|  OK|IAE0|
|79062006110497|  1|CDG|MAD| AF| AF|26.02|USD|  1|  Y|   Y|1300|150724|  SA|INA0|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
only showing top 5 rows



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [ ]:
session.read.json
session.read.parquet

### Basic operations with DataFrames

In [50]:
df.show(2)

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1| mecanic|
+---+--------+
only showing top 2 rows



In [51]:
df.take(2)

[Row(id=0, position='manager'), Row(id=1, position='mecanic')]

In [52]:
df.schema

StructType(List(StructField(id,LongType,true),StructField(position,StringType,true)))

In [54]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)



### Filtering and selecting

Syntax inspired in SQL.

In [55]:
df.select('id')

DataFrame[id: bigint]

In [58]:
df.select(df['id']).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
+---+



If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [61]:
filtered = df.filter(df['id']>5)
filtered.show()

+---+--------+
| id|position|
+---+--------+
|  6| mecanic|
|  7| mecanic|
|  8| manager|
|  9| mecanic|
| 10| manager|
| 11| manager|
+---+--------+



That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

In [62]:
filtered = df.filter('id'>5)

TypeError: '>' not supported between instances of 'str' and 'int'

In [63]:
df['id']

Column<b'id'>

`where` is exactly synonimous with `filter`

In [64]:
whered = df.where(df['id']>5)
whered.show()

+---+--------+
| id|position|
+---+--------+
|  6| mecanic|
|  7| mecanic|
|  8| manager|
|  9| mecanic|
| 10| manager|
| 11| manager|
+---+--------+



A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

In [65]:
col = df['id']
col.collect()

TypeError: 'Column' object is not callable

#### Exercise

Extract all employee ids which correspond to managers

In [72]:
df.where(df['position']=='manager').select(df['id']).show()

+---+
| id|
+---+
|  0|
|  3|
|  8|
| 10|
| 11|
+---+



In [69]:
df[df['position']=='manager'].select(df['id']).show()

+---+
| id|
+---+
|  0|
|  3|
|  8|
| 10|
| 11|
+---+



In [70]:
df.select(df['position']=='manager').show()

+--------------------+
|(position = manager)|
+--------------------+
|                true|
|               false|
|               false|
|                true|
|               false|
|               false|
|               false|
|               false|
|                true|
|               false|
|                true|
|                true|
+--------------------+



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we can not assign to them. We need to create new DataFrames with the appropriate columns.

In [71]:
df['anewcolumn']=df['id']*10

TypeError: 'DataFrame' object does not support item assignment

In [73]:
df2 = df.withColumn('anewcolumn',df['id']*100)

### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [74]:
from pyspark.sql import functions
functions.sqrt(df['id'])

Column<b'SQRT(id)'>

In [75]:
df.select(functions.sqrt('id')).show(3)

+------------------+
|          SQRT(id)|
+------------------+
|               0.0|
|               1.0|
|1.4142135623730951|
+------------------+
only showing top 3 rows



In [76]:
df3 = df.select('id',
                'position',
                functions.sqrt('id'))

In [77]:
import math
math.log1p(0)

0.0

In [78]:
df3 = df.select('id',
                'position',
                math.log1p('id'))
df3.show()

TypeError: must be real number, not str

This errors out because 

```python
math.log1p
```

is not a udf: it doesn't know how to work with strings or Column objects:

But we can transform it into a udf:

In [79]:
udf_log1p = functions.udf(math.log1p)

In [80]:
udf_log1p(col)

Column<b'log1p(id)'>

We can do the same with any function we dream up:

In [83]:
def odd_letters(word):
    return word[::2]
odd_letters('manager')

'mngr'

In [84]:
odd_udf = functions.udf(odd_letters)
odd_udf(col)

Column<b'odd_letters(id)'>

If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [85]:
df3 = df.select('id',
               'position',
               udf_log1p('id'))
df3.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- log1p(id): string (nullable = true)



In [86]:
udf_log1p_typed = functions.udf(math.log1p, types.FloatType())

In [87]:
df3 = df.select('id',
               'position',
               udf_log1p_typed('id'))
df3.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- log1p(id): float (nullable = true)



Think about this function: what is its return type?

In [89]:
functions.udf(lambda x: x + x)

<function __main__.<lambda>(x)>

#### Exercise: 

Create a 'salary' field in our df. make it 30000 for mechanics, 40000 for salespeople and 70000 for managers.



In [92]:
def salary_from(position):
    salaries={'manager':70000,'sales':40000,'mechanic':30000}
    return salaries.get(position)
    #return salaries[position]

salary_from('manager')

70000

In [94]:
salary_udf = functions.udf(salary_from)

If we have a column that is not the desired type, we can convert it with `cast`.

In [96]:
exercise_result = df.withColumn('salary',salary_udf('position').cast(types.IntegerType()))

In [97]:
exercise_result.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- salary: integer (nullable = true)



### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [98]:
exercise_result.stat.corr('id','salary')

0.18404118228316377

In [100]:
exercise_result.stat.cov('id','salary')

23181.81818181818

### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

In [104]:
location_udf = functions.udf(lambda: random.choice(['Madrid','Barcelona']))

In [105]:
df4 = exercise_result.withColumn('location',location_udf()).cache()

In [106]:
df4.show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1| mecanic|  null|   Madrid|
|  2| mecanic|  null|   Madrid|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|   Madrid|
|  5| mecanic|  null|   Madrid|
|  6| mecanic|  null|Barcelona|
|  7| mecanic|  null|   Madrid|
|  8| manager| 70000|   Madrid|
|  9| mecanic|  null|Barcelona|
| 10| manager| 70000|   Madrid|
| 11| manager| 70000|   Madrid|
+---+--------+------+---------+



In [107]:
df4.crosstab('position','location').show()

+-----------------+---------+------+
|position_location|Barcelona|Madrid|
+-----------------+---------+------+
|            sales|        0|     1|
|          manager|        2|     3|
|          mecanic|        2|     4|
+-----------------+---------+------+



### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [108]:
df4.groupBy('location')

GroupedData has several aggregation functions defined:

In [109]:
df4.groupBy('location').mean().show()

+---------+-------+-----------+
| location|avg(id)|avg(salary)|
+---------+-------+-----------+
|   Madrid|    6.0|    62500.0|
|Barcelona|    4.5|    70000.0|
+---------+-------+-----------+



In [110]:
df4.groupBy('location').mean('salary').show()

+---------+-----------+
| location|avg(salary)|
+---------+-----------+
|   Madrid|    62500.0|
|Barcelona|    70000.0|
+---------+-----------+



In [113]:
stats = df4.groupBy('location').agg(functions.mean('salary').alias('average'),
                                    functions.stddev_pop('salary').alias('stddev'),
                                    functions.count('salary'))
stats.show()

+---------+-------+-----------------+-------------+
| location|average|           stddev|count(salary)|
+---------+-------+-----------------+-------------+
|   Madrid|62500.0|12990.38105676658|            4|
|Barcelona|70000.0|              0.0|            2|
+---------+-------+-----------------+-------------+



We can do several aggregations in a single step, with a number of different syntaxes:

### Intersections

Ver much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

In [116]:
random.seed(42)
data = list(zip([10,11,9,10,9],
               [5000,1000,2000,0,1000],
               [random.choice(['Madrid','Barcelomna','Sevilla']) for _ in range(5)]))

In [117]:
data

[(10, 5000, 'Sevilla'),
 (11, 1000, 'Madrid'),
 (9, 2000, 'Madrid'),
 (10, 0, 'Sevilla'),
 (9, 1000, 'Barcelomna')]

In [118]:
new_df = session.createDataFrame(data,schema=['id','bonus','location'])

In [120]:
new_df.show()

+---+-----+----------+
| id|bonus|  location|
+---+-----+----------+
| 10| 5000|   Sevilla|
| 11| 1000|    Madrid|
|  9| 2000|    Madrid|
| 10|    0|   Sevilla|
|  9| 1000|Barcelomna|
+---+-----+----------+



Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
session.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```



In [119]:
df4.show()

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1| mecanic|  null|   Madrid|
|  2| mecanic|  null|   Madrid|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|   Madrid|
|  5| mecanic|  null|   Madrid|
|  6| mecanic|  null|Barcelona|
|  7| mecanic|  null|   Madrid|
|  8| manager| 70000|   Madrid|
|  9| mecanic|  null|Barcelona|
| 10| manager| 70000|   Madrid|
| 11| manager| 70000|   Madrid|
+---+--------+------+---------+



In [121]:
df4.join(new_df).show()

AnalysisException: 'Detected implicit cartesian product for INNER join between logical plans\nInMemoryRelation [id#13L, position#14, salary#455, location#539], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)\n   +- *(1) Project [id#13L, position#14, cast(pythonUDF0#544 as int) AS salary#455, pythonUDF1#545 AS location#539]\n      +- BatchEvalPython [salary_from(position#14), <lambda>()], [id#13L, position#14, pythonUDF0#544, pythonUDF1#545]\n         +- Scan ExistingRDD[id#13L,position#14]\nand\nLogicalRDD [id#970L, bonus#971L, location#972], false\nJoin condition is missing or trivial.\nEither: use the CROSS JOIN syntax to allow cartesian products between these\nrelations, or: enable implicit cartesian products by setting the configuration\nvariable spark.sql.crossJoin.enabled=true;'

In [122]:
df4.join(new_df,on='id').show()

+---+--------+------+---------+-----+----------+
| id|position|salary| location|bonus|  location|
+---+--------+------+---------+-----+----------+
| 10| manager| 70000|   Madrid| 5000|   Sevilla|
| 11| manager| 70000|   Madrid| 1000|    Madrid|
|  9| mecanic|  null|Barcelona| 2000|    Madrid|
| 10| manager| 70000|   Madrid|    0|   Sevilla|
|  9| mecanic|  null|Barcelona| 1000|Barcelomna|
+---+--------+------+---------+-----+----------+



In [123]:
df4.join(new_df,on='id',how='left').show()

+---+--------+------+---------+-----+----------+
| id|position|salary| location|bonus|  location|
+---+--------+------+---------+-----+----------+
|  0| manager| 70000|Barcelona| null|      null|
|  7| mecanic|  null|   Madrid| null|      null|
|  6| mecanic|  null|Barcelona| null|      null|
|  9| mecanic|  null|Barcelona| 2000|    Madrid|
|  9| mecanic|  null|Barcelona| 1000|Barcelomna|
|  5| mecanic|  null|   Madrid| null|      null|
|  1| mecanic|  null|   Madrid| null|      null|
| 10| manager| 70000|   Madrid| 5000|   Sevilla|
| 10| manager| 70000|   Madrid|    0|   Sevilla|
|  3| manager| 70000|Barcelona| null|      null|
|  8| manager| 70000|   Madrid| null|      null|
| 11| manager| 70000|   Madrid| 1000|    Madrid|
|  2| mecanic|  null|   Madrid| null|      null|
|  4|   sales| 40000|   Madrid| null|      null|
+---+--------+------+---------+-----+----------+



In [124]:
df4.join(new_df,on=new_df['id']==df4['id'],how='left').show()

+---+--------+------+---------+----+-----+----------+
| id|position|salary| location|  id|bonus|  location|
+---+--------+------+---------+----+-----+----------+
|  0| manager| 70000|Barcelona|null| null|      null|
|  7| mecanic|  null|   Madrid|null| null|      null|
|  6| mecanic|  null|Barcelona|null| null|      null|
|  9| mecanic|  null|Barcelona|   9| 2000|    Madrid|
|  9| mecanic|  null|Barcelona|   9| 1000|Barcelomna|
|  5| mecanic|  null|   Madrid|null| null|      null|
|  1| mecanic|  null|   Madrid|null| null|      null|
| 10| manager| 70000|   Madrid|  10| 5000|   Sevilla|
| 10| manager| 70000|   Madrid|  10|    0|   Sevilla|
|  3| manager| 70000|Barcelona|null| null|      null|
|  8| manager| 70000|   Madrid|null| null|      null|
| 11| manager| 70000|   Madrid|  11| 1000|    Madrid|
|  2| mecanic|  null|   Madrid|null| null|      null|
|  4|   sales| 40000|   Madrid|null| null|      null|
+---+--------+------+---------+----+-----+----------+



In [125]:
df4.join(new_df,on=['id','location'],how='left').show()

+---+---------+--------+------+-----+
| id| location|position|salary|bonus|
+---+---------+--------+------+-----+
|  3|Barcelona| manager| 70000| null|
|  2|   Madrid| mecanic|  null| null|
|  0|Barcelona| manager| 70000| null|
|  8|   Madrid| manager| 70000| null|
|  7|   Madrid| mecanic|  null| null|
|  6|Barcelona| mecanic|  null| null|
|  4|   Madrid|   sales| 40000| null|
|  9|Barcelona| mecanic|  null| null|
| 10|   Madrid| manager| 70000| null|
|  5|   Madrid| mecanic|  null| null|
| 11|   Madrid| manager| 70000| 1000|
|  1|   Madrid| mecanic|  null| null|
+---+---------+--------+------+-----+



#### Digression

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

We can erase it with `unpersist`

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

1) Calculate the mean and std of salary for each location

2) Annotate each employee with the stats corresponding to their location

3) Calculate the z-score

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

Once registered, we can perform queries as complex as we want.

#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

## Writing out


#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

2) Total ticket amounts per origin

3) Top 10 Airlines by average amount



## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf